In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import umap
import umap.plot

warnings.filterwarnings('ignore')

c:\Users\이정진\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = os.path.join(os.getcwd(),'data')
files = os.listdir(data_path)[:-1]
files

['aisles.csv',
 'departments.csv',
 'orders.csv',
 'order_products__prior.csv',
 'order_products__train.csv',
 'products.csv']

In [3]:
aisles = pd.read_csv(os.path.join(data_path,files[0]), engine='pyarrow') # (134, 2)
departments = pd.read_csv(os.path.join(data_path,files[1]), engine='pyarrow') # (21, 2)
# orders = pd.read_csv(os.path.join(data_path,files[2]), engine='pyarrow') # (3421083, 7)
# orders_products_prior = pd.read_csv(os.path.join(data_path,files[3]),engine='pyarrow') # (32434489, 4)
# orders_products_train = pd.read_csv(os.path.join(data_path,files[4]),engine='pyarrow') # (1384617, 4)
products = pd.read_csv(os.path.join(data_path,files[5]),engine='pyarrow') # (49688, 4)

In [4]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


* aisles - 통로(코너, 냉동 department의 어류 이런 세부종류?) aisleid, aisle
* departments - department종류(냉동식품 등)  department id, department
* orders - userid, eval_set(prior, train, test), 주문번호, 주문요일?, 주문시간대, since prior order?
* order_products_prior - orderid, 상품id, add to cart order, reorderd(재주문 여부, none은 0으로)
* order_products_train - 
* products - 상품id, 상품명, aisleid, departmentid

# 결측치 여부

In [ ]:
tables = [aisles, departments, orders, orders_products_prior, orders_products_train, products]

In [ ]:
for t in tables:
    print(t.isnull().sum())

days_since_prior_order에만 결측치 존재

In [ ]:
prior = orders[orders['eval_set']== 'prior']
train = orders[orders['eval_set'] == 'train']
test = orders[orders['eval_set'] == 'test']

In [ ]:
print(prior.shape)
print(train.shape)
print(test.shape)
print(orders.shape)


In [ ]:
del train['eval_set'], test['eval_set'], prior['eval_set']

In [ ]:
train.head()

In [ ]:
pd.merge(orders_products_train, train, how='inner')

In [ ]:
orders_products_train.shape, train.shape

In [ ]:
temp = orders_products_train.set_index('order_id')
orders_train = pd.merge(temp, train, on='order_id')
orders_train.head()

In [ ]:
temp = orders_products_prior.set_index('order_id')
orders_prior = pd.merge(temp, prior, on='order_id')
orders_prior.head() # 1분걸림

In [ ]:
total_orders = pd.concat([orders_prior, orders_train])
total_orders.to_csv('total_orders.parquet', index=False)

orders, orders_products_prior, orders_products_train을 모두 합쳐줬다.(test 제외)

In [ ]:
# total_orders = pd.read_csv('total_orders.csv',engine='pyarrow') # 1분 15초 

In [ ]:
total_orders = pd.read_parquet('total_orders.parquet',engine='fastparquet') # 5초
temp = pd.merge(total_orders,products , on='product_id')
del total_orders # 메모리..
temp = pd.merge(temp, aisles, on='aisle_id')
temp = pd.merge(temp, departments, on='department_id')
temp.to_parquet('total_orders.parquet', index=False)

In [ ]:
temp.to_parquet('total_orders.parquet', index=False)
del temp

In [ ]:
pd.options.display.max_rows = 10
practice.set_index('user_id','order_id')[:100]

In [ ]:
df = practice[['order_id','product_id','user_id']]

In [ ]:
df[df['user_id']=='10']

In [ ]:
df2 = df.groupby(['user_id','order_id']).product_id.apply(list).reset_index(name='order_list')
df2

In [ ]:
df2.sort_values('order_id')

In [ ]:
df3 = pd.merge(practice,products , on='product_id')

In [ ]:
df4 = pd.merge(df3, aisles, on='aisle_id')
df5 = pd.merge(df4, departments, on='department_id')

In [ ]:
df5

재구매율 높은 고객을 보기 위해 재주문건수/주문건수 로 만들어보겠음

In [ ]:
df5.sort_values(['user_id','order_id'])

In [ ]:
b=df5.groupby(['user_id']).agg({'reordered':'sum', 'user_id':'count'}).rename(columns={'user_id':'count'}).reset_index()

In [ ]:
b['reorder_ratio'] = b['reordered'] / b['count']

In [ ]:
sns.histplot(b['reorder_ratio'])

In [ ]:
b['class'] = pd.qcut(b['reorder_ratio'],[0,0.25,0.5,0.75,1], labels=[4,3,2,1])

In [ ]:
b['class'].value_counts().plot(kind='bar')

In [ ]:
tab1 = pd.crosstab(df4['user_id'], df4['aisle'])

In [ ]:
tab2 = pd.crosstab(df5['user_id'], df5['department'])

In [ ]:
tab1

In [ ]:
tab2

In [ ]:
label_table = b[['user_id','class']]
label = pd.merge(tab2,label_table, on='user_id')['class']

In [ ]:
label

# Umap

In [ ]:
from sklearn.preprocessing import StandardScaler

scaled_tab = StandardScaler().fit_transform(tab2)

In [ ]:
embedding = umap.UMAP(n_neighbors=10,
                      min_dist=0.2,
                      metric='correlation').fit(scaled_tab)

In [ ]:
embedding

In [ ]:
umap.plot.points(embedding, label.astype('int'), theme='fire')

잘 구분이 되지 않아서 라벨 범위를 좁히기로 했다.

In [ ]:
b['class'] = pd.qcut(b['reorder_ratio'],[0,0.2,0.7,1], labels=[3,2,1])

In [ ]:
b['class'].value_counts().plot(kind='bar')

In [ ]:
label_table = b[['user_id','class']]
label = pd.merge(tab2,label_table, on='user_id')['class']

In [ ]:
umap.plot.points(embedding, label, theme='fire')